<a href="https://colab.research.google.com/github/jhk990602/datapractice/blob/main/lgaimers%5Bautogluon%5D_%EC%A0%84%EC%B2%98%EB%A6%ACx2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://dacon.io/codeshare/7803

In [1]:
!pip install autogluon

In [2]:
import os
import pandas as pd
import random
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag

import warnings
warnings.filterwarnings("ignore")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(50)

data = pd.read_csv('/content/drive/MyDrive/lg/train.csv')

In [3]:
data.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


In [4]:
from sklearn.model_selection import train_test_split
# 데이터를 로드하고, 훈련 데이터와 테스트 데이터로 분리합니다.

train_data, test_data = train_test_split(data, test_size=0.2)

In [5]:
train_data = TabularDataset(train_data)
test_data = TabularDataset(test_data)

label = 'is_converted'
eval_metric = 'f1'
time_limit = 5000

In [6]:
#excluded_model_types = ['CAT']
predictor = TabularPredictor(
    label=label, problem_type='binary', eval_metric=eval_metric
).fit(train_data,
      presets='best_quality',
      num_stack_levels=3,
      #excluded_model_types = excluded_model_types,
      time_limit=time_limit, num_gpus=1)

No path specified. Models will be saved in: "AutogluonModels/ag-20240205_174942"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 5000 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240205_174942/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 1250s
AutoGluon will save models to "AutogluonModels/ag-20240205_174942/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  1

In [7]:
print(predictor.leaderboard(silent = True))

                      model  score_val eval_metric  pred_time_val  \
0       WeightedEnsemble_L4   0.865120          f1      64.210718   
1       WeightedEnsemble_L5   0.863879          f1      69.255343   
2           LightGBM_BAG_L4   0.863717          f1      69.169780   
3            XGBoost_BAG_L3   0.862290          f1      58.639211   
4            XGBoost_BAG_L4   0.861988          f1      69.323979   
5    NeuralNetFastAI_BAG_L3   0.861271          f1      59.548290   
6         LightGBMXT_BAG_L4   0.860969          f1      69.206576   
7       WeightedEnsemble_L3   0.860189          f1      52.359881   
8   RandomForestEntr_BAG_L4   0.859989          f1      70.618618   
9   RandomForestEntr_BAG_L3   0.859859          f1      59.763744   
10     LightGBMLarge_BAG_L3   0.859680          f1      58.634974   
11           XGBoost_BAG_L2   0.859601          f1      47.362491   
12        LightGBMXT_BAG_L3   0.859245          f1      58.477340   
13  RandomForestGini_BAG_L3   0.85

In [8]:
predictor.feature_importance(train_data) #original

These features in provided data are not utilized by the predictor and will be ignored: ['customer_country.1']
Computing feature importance via permutation shuffling for 27 features using 5000 rows with 5 shuffle sets...
	2551.55s	= Expected runtime (510.31s per shuffle set)
	1382.75s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
customer_idx,0.527091,0.033420,0.000002,5,0.595903,0.458280
lead_owner,0.120435,0.007767,0.000002,5,0.136428,0.104441
lead_desc_length,0.033035,0.005526,0.000091,5,0.044414,0.021657
business_subarea,0.022372,0.002563,0.000020,5,0.027648,0.017096
customer_type,0.022235,0.002224,0.000012,5,0.026814,0.017656
expected_timeline,0.015276,0.006546,0.003217,5,0.028754,0.001799
response_corporate,0.008458,0.004335,0.006016,5,0.017383,-0.000467
customer_country,0.008314,0.003601,0.003344,5,0.015730,0.000899
historical_existing_cnt,0.007430,0.003527,0.004620,5,0.014693,0.000167
com_reg_ver_win_rate,0.005038,0.001000,0.000177,5,0.007096,0.002979


In [9]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data, model=model_to_use)

submission = pd.read_csv('/content/drive/MyDrive/lg/submission.csv')

submission["is_converted"] = model_pred
submission.to_csv('/content/drive/MyDrive/lg/autogluon_submission2.csv', index=False, encoding="utf-8")